In [ ]:
#Dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
import time
import pymongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn) 
db = client.goodreads_db
collection = db.items

In [ ]:
import json
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('hYXUmMhIZHj11RqZ0B4ltxyIlp5VrpGof7WNbPHiiVH0')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.us-south.tone-analyzer.watson.cloud.ibm.com/instances/3a2563d5-be5b-44db-a536-84cb59e18ead')

In [ ]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = 'https://www.goodreads.com/genres/most_read/childrens'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')


In [ ]:
book_boxes = soup.find_all('div', class_="leftAlignedImage bookBox")
links = []

for box in book_boxes:
    
    link = box.find('a', href=True)['href']
    links.append('https://www.goodreads.com' + link)

In [ ]:
'''
for link in links:
    browser.visit(link)
    time.sleep(2)
    
    results = soup.find_all('span')
    
    print(results)
    
    
    for result in results:
        try:
            review = result.find('span').text
            print(review)
        except:
            print("N/A")
    
    browser.back()
    time.sleep(2)

'''

In [ ]:
import requests

for link in links:
    # Retrieve page with the requests module
    response = requests.get(link)
    # Create BeautifulSoup object; parse with 'lxml'
    soup = BeautifulSoup(response.text, 'lxml')
    
    title = soup.find('h1', id="bookTitle").text
    
    author = soup.find('a', class_='authorName').text
    
    results = soup.find_all('div', class_='review')

    for result in results:
        try:
            rating = result.find('span', class_='staticStars notranslate')['title']
            
            review = result.find('span',class_="readable").text
            
            tone_analysis = tone_analyzer.tone(
                                                {'text': review},
                                                content_type='text/plain',
                                                sentences = False).get_result()
            
            tone1 = tone_analysis["document_tone"]["tones"][0]["tone_name"]
            
            tone2 = tone_analysis["document_tone"]["tones"][0]["tone_name"]
            
            if (rating and review and tone_analysis and tone1 and tone2):
                
            
                print(title)
                print(author)
                print(rating)
                print(review)
                print(tone1)
                print(tone2)
                
                post = {
                    'rating': rating,
                    'review': review,
                    'tone_analysis': tone_analysis,
                    'tone1': tone1,
                    'tone2': tone2
                }
                
                collection.insert_one(post)

        except Exception as e:
            print("N/A")
    break